###Set up paths 

In [0]:
RAW = "/Volumes/demo/bronze/movie/imdb_movies.csv"      # upload your CSV here (Data → Add → Upload)
OUT = "/Volumes/demo/bronze/movi/out"             # where we’ll write results


In [0]:
df = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv(RAW))

display(df.limit(5))
print(df.columns)
df.printSchema()


names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Creed has been thriving in both his career and family life. When a childhood friend and former boxing prodigy, Damien Anderson, resurfaces after serving a long sentence in prison, he is eager to prove that he deserves his shot in the ring. The face-off between former friends is more than just a fight. To settle the score, Adonis must put his future on the line to battle Damien — a fighter who has nothing to lose.","Michael B. Jordan, Adonis Creed, Tessa Thompson, Bianca Taylor, Jonathan Majors, Damien Anderson, Wood Harris, Tony 'Little Duke' Evers, Phylicia Rashād, Mary Anne Creed, Mila Davis-Kent, Amara Creed, Florian Munteanu, Viktor Drago, José Benavidez Jr., Felix Chavez, Selenis Leyva, Laura Chavez",Creed III,Released,English,75000000.0,271616668.0,AU
Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action","Set more than a decade after the events of the first film, learn the story of the Sully family (Jake, Neytiri, and their kids), the trouble that follows them, the lengths they go to keep each other safe, the battles they fight to stay alive, and the tragedies they endure.","Sam Worthington, Jake Sully, Zoe Saldaña, Neytiri, Sigourney Weaver, Kiri / Dr. Grace Augustine, Stephen Lang, Colonel Miles Quaritch, Kate Winslet, Ronal, Cliff Curtis, Tonowari, Joel David Moore, Norm Spellman, CCH Pounder, Mo'at, Edie Falco, General Frances Ardmore",Avatar: The Way of Water,Released,English,460000000.0,2316794914.0,AU
The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main, Brooklyn plumbers—and brothers—Mario and Luigi are transported down a mysterious pipe and wander into a magical new world. But when the brothers are separated, Mario embarks on an epic quest to find Luigi.","Chris Pratt, Mario (voice), Anya Taylor-Joy, Princess Peach (voice), Charlie Day, Luigi (voice), Jack Black, Bowser (voice), Keegan-Michael Key, Toad (voice), Seth Rogen, Donkey Kong (voice), Fred Armisen, Cranky Kong (voice), Kevin Michael Richardson, Kamek (voice), Sebastian Maniscalco, Spike (voice)",The Super Mario Bros. Movie,Released,English,100000000.0,724459031.0,AU
Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three mummies end up in present-day London and embark on a wacky and hilarious journey in search of an old ring belonging to the Royal Family, stolen by ambitious archaeologist Lord Carnaby.","Óscar Barberán, Thut (voice), Ana Esther Alborg, Nefer (voice), Luis Pérez Reina, Carnaby (voice), María Luisa Solá, Madre (voice), Jaume Solà, Sekhem (voice), José Luis Mediavilla, Ed (voice), José Javier Serrano Rodríguez, Danny (voice), Aleix Estadella, Dennis (voice), María Moscardó, Usi (voice)",Momias,Released,"Spanish, Castilian",12300000.0,34200000.0,AU
Supercell,03/17/2023,61.0,Action,"Good-hearted teenager William always lived in hope of following in his late father’s footsteps and becoming a storm chaser. His father’s legacy has now been turned into a storm-chasing tourist business, managed by the greedy and reckless Zane Rogers, who is now using William as the main attraction to lead a group of unsuspecting adventurers deep into the eye of the most dangerous supercell ever seen.","Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quinn Brody, Daniel Diemer, William Brody, Jordan Kristine Seamón, Harper Hunter, Alec Baldwin, Zane Rogers, Richard Gunn, Bill Brody, Praya Lundberg, Amy, Johnny Wactor, Martin, Anjul Nigam, Ramesh",Supercell,Released,English,77000000.0,340941958.6,US


['names', 'date_x', 'score', 'genre', 'overview', 'crew', 'orig_title', 'status', 'orig_lang', 'budget_x', 'revenue', 'country']
root
 |-- names: string (nullable = true)
 |-- date_x: string (nullable = true)
 |-- score: double (nullable = true)
 |-- genre: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- crew: string (nullable = true)
 |-- orig_title: string (nullable = true)
 |-- status: string (nullable = true)
 |-- orig_lang: string (nullable = true)
 |-- budget_x: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- country: string (nullable = true)



In [0]:
from pyspark.sql import functions as F

df_raw = (spark.read
          .option("header", True)
          .option("inferSchema", True)
          .csv(RAW))

# 1) trim spaces
d = F.trim(F.col("date_x"))

# 2) parse based on separator
date_slash = F.to_date(d, "MM/dd/yyyy")   # e.g., 12/15/2022
date_dash  = F.to_date(d, "dd-MM-yyyy")   # e.g., 03-02-2023

parsed = (
    F.when(F.instr(d, "/") > 0, date_slash)
     .when(F.instr(d, "-") > 0, date_dash)
     .otherwise(F.lit(None).cast("date"))
)

# 3) fallback year if still null (e.g., weird strings)
year_fallback = F.regexp_extract(d, r"(\d{4})", 1).cast("int")

df = df_raw.select(
    F.coalesce(F.col("names"), F.col("orig_title")).alias("title"),
    F.col("genre").alias("genres"),
    F.coalesce(F.year(parsed), year_fallback).alias("release_year"),
    F.col("score").cast("double").alias("rating"),
    F.col("country")
)

display(df.limit(10))
print("Null release_year count:", df.filter(F.col("release_year").isNull()).count())


title,genres,release_year,rating,country
Creed III,"Drama, Action",2023,73.0,AU
Avatar: The Way of Water,"Science Fiction, Adventure, Action",2022,78.0,AU
The Super Mario Bros. Movie,"Animation, Adventure, Family, Fantasy, Comedy",2023,76.0,AU
Mummies,"Animation, Comedy, Family, Adventure, Fantasy",2023,70.0,AU
Supercell,Action,2023,61.0,US
Cocaine Bear,"Thriller, Comedy, Crime",2023,66.0,AU
John Wick: Chapter 4,"Action, Thriller, Crime",2023,80.0,AU
Puss in Boots: The Last Wish,"Animation, Family, Fantasy, Adventure, Comedy",2022,83.0,AU
Attack on Titan,"Action, Science Fiction",2022,59.0,US
The Park,"Action, Drama, Horror, Science Fiction, Thriller",2023,58.0,US


Null release_year count: 0


Cell 3 — Clean + explode genres

In [0]:
df = df.where(F.col("title").isNotNull() & F.col("release_year").isNotNull())
df = df.withColumn("genres", F.coalesce(F.col("genres"), F.lit("Unknown")))
df = df.withColumn("genres", F.regexp_replace(F.col("genres"), r"\s+", ""))

df_exploded = df.withColumn("genre", F.explode(F.split("genres", r"[|,]")))
display(df_exploded.limit(10))


title,genres,release_year,rating,country,genre
Creed III,"Drama, Action",2023,73.0,AU,Drama
Creed III,"Drama, Action",2023,73.0,AU,Action
Avatar: The Way of Water,"ScienceFiction, Adventure, Action",2022,78.0,AU,ScienceFiction
Avatar: The Way of Water,"ScienceFiction, Adventure, Action",2022,78.0,AU,Adventure
Avatar: The Way of Water,"ScienceFiction, Adventure, Action",2022,78.0,AU,Action
The Super Mario Bros. Movie,"Animation, Adventure, Family, Fantasy, Comedy",2023,76.0,AU,Animation
The Super Mario Bros. Movie,"Animation, Adventure, Family, Fantasy, Comedy",2023,76.0,AU,Adventure
The Super Mario Bros. Movie,"Animation, Adventure, Family, Fantasy, Comedy",2023,76.0,AU,Family
The Super Mario Bros. Movie,"Animation, Adventure, Family, Fantasy, Comedy",2023,76.0,AU,Fantasy
The Super Mario Bros. Movie,"Animation, Adventure, Family, Fantasy, Comedy",2023,76.0,AU,Comedy


Cell 4 — Aggregations

In [0]:
# Top genres by count
genres_pop = (df_exploded.groupBy("genre")
              .count()
              .orderBy(F.desc("count")))

# Average rating by release year
rating_by_year = (df_exploded
                  .where(F.col("rating").isNotNull())
                  .groupBy("release_year")
                  .agg(F.avg("rating").alias("avg_rating"))
                  .orderBy("release_year"))

display(genres_pop.limit(10))
display(rating_by_year.limit(10))


genre,count
Thriller,2028
Drama,1944
Drama,1868
Comedy,1565
Action,1565
Comedy,1378
Adventure,1319
Action,1187
Romance,1162
Fantasy,1118


release_year,avg_rating
1903,63.0
1907,80.0
1915,61.0
1920,80.0
1923,74.6
1925,74.8
1928,74.33333333333333
1929,75.0
1930,78.0
1931,77.58333333333333


Cell 5 — Save + Query

In [0]:
# Save to DBFS
genres_pop.write.mode("overwrite").parquet(f"{OUT}/genres_pop")
rating_by_year.write.mode("overwrite").parquet(f"{OUT}/rating_by_year")

# Query via SparkSQL
genres_pop.createOrReplaceTempView("genres_pop")
display(spark.sql("SELECT * FROM genres_pop ORDER BY count DESC LIMIT 10"))

print("✅ Saved outputs under:", OUT)
